# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about vision APIs for vision-language models and embedding APIs for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0"
)

wait_for_server("http://localhost:30000")

[2025-01-02 17:27:58] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, return_token_ids=False, host='0.0.0.0', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, random_seed=477756401, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_

[2025-01-02 17:28:12 TP0] Init torch distributed begin.


[2025-01-02 17:28:13 TP0] Load weight begin. avail mem=78.81 GB


[2025-01-02 17:28:14 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

100%|██████████| 23/23 [00:05<00:00,  4.45it/s]
[2025-01-02 17:28:23 TP0] Capture cuda graph end. Time elapsed: 5.18 s


[2025-01-02 17:28:23 TP0] max_total_num_tokens=444500, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2025-01-02 17:28:23] INFO:     Started server process [1673743]
[2025-01-02 17:28:23] INFO:     Waiting for application startup.
[2025-01-02 17:28:23] INFO:     Application startup complete.
[2025-01-02 17:28:23] INFO:     Uvicorn running on http://0.0.0.0:30000 (Press CTRL+C to quit)


[2025-01-02 17:28:24] INFO:     127.0.0.1:34266 - "GET /v1/models HTTP/1.1" 200 OK


[2025-01-02 17:28:24] INFO:     127.0.0.1:34268 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-01-02 17:28:24 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-02 17:28:25] INFO:     127.0.0.1:34274 - "POST /generate HTTP/1.1" 200 OK
[2025-01-02 17:28:25] The server is fired up and ready to roll!


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-01-02 17:28:29 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, cache hit rate: 2.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-02 17:28:29 TP0] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 6.50, #queue-req: 0
[2025-01-02 17:28:29] INFO:     127.0.0.1:37328 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-01-02 17:28:29 TP0] Prefill batch. #new-seq: 1, #new-token: 51, #cached-token: 25, cache hit rate: 20.63%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-02 17:28:29 TP0] frequency_penalty, presence_penalty, and repetition_penalty are not supported when using the default overlap scheduler. They will be ignored. Please add `--disable-overlap` when launching the server if you need these features. The speed will be slower in that case.


[2025-01-02 17:28:30 TP0] Decode batch. #running-req: 1, #token: 106, token usage: 0.00, gen throughput (token/s): 126.06, #queue-req: 0


[2025-01-02 17:28:30 TP0] Decode batch. #running-req: 1, #token: 146, token usage: 0.00, gen throughput (token/s): 141.19, #queue-req: 0


[2025-01-02 17:28:30 TP0] Decode batch. #running-req: 1, #token: 186, token usage: 0.00, gen throughput (token/s): 140.59, #queue-req: 0
[2025-01-02 17:28:30] INFO:     127.0.0.1:37328 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-01-02 17:28:30] INFO:     127.0.0.1:37328 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-01-02 17:28:30 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 30, cache hit rate: 33.73%, token usage: 0.00, #running-req: 0, #queue-req: 0
It looks like you're starting a test

. Are you ready to begin?

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-01-02 17:28:30 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 32.57%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-02 17:28:31 TP0] Decode batch. #running-req: 1, #token: 15, token usage: 0.00, gen throughput (token/s): 130.74, #queue-req: 0


[2025-01-02 17:28:31 TP0] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 146.23, #queue-req: 0
[2025-01-02 17:28:31] INFO:     127.0.0.1:37328 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-01-02 17:28:31 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 31.35%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-02 17:28:31 TP0] frequency_penalty, presence_penalty, and repetition_penalty are not supported when using the default overlap scheduler. They will be ignored. Please add `--disable-overlap` when launching the server if you need these features. The speed will be slower in that case.


[2025-01-02 17:28:31 TP0] Decode batch. #running-req: 1, #token: 32, token usage: 0.00, gen throughput (token/s): 138.70, #queue-req: 0


[2025-01-02 17:28:31 TP0] Decode batch. #running-req: 1, #token: 72, token usage: 0.00, gen throughput (token/s): 145.21, #queue-req: 0


[2025-01-02 17:28:32 TP0] Decode batch. #running-req: 1, #token: 112, token usage: 0.00, gen throughput (token/s): 143.49, #queue-req: 0


[2025-01-02 17:28:32 TP0] Decode batch. #running-req: 1, #token: 152, token usage: 0.00, gen throughput (token/s): 142.42, #queue-req: 0
[2025-01-02 17:28:32] INFO:     127.0.0.1:37328 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)
You can specify a JSON schema, [regular expression](https://en.wikipedia.org/wiki/Regular_expression) or [EBNF](https://en.wikipedia.org/wiki/Extended_Backus%E2%80%93Naur_form) to constrain the model output. The model output will be guaranteed to follow the given constraints. Only one constraint parameter (`json_schema`, `regex`, or `ebnf`) can be specified for a request.

SGLang supports two grammar backends:

- [Outlines](https://github.com/dottxt-ai/outlines) (default): Supports JSON schema and regular expression constraints.
- [XGrammar](https://github.com/mlc-ai/xgrammar): Supports JSON schema and EBNF constraints.
  - XGrammar currently uses the [GGML BNF format](https://github.com/ggerganov/llama.cpp/blob/master/grammars/README.md)

Initialize the XGrammar backend using `--grammar-backend xgrammar` flag
```bash
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
--port 30000 --host 0.0.0.0 --grammar-backend [xgrammar|outlines] # xgrammar or outlines (default: outlines)
```

### JSON

In [7]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=128,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(response.choices[0].message.content)

Compiling FSM index for all state transitions:   0%|          | 0/38 [00:00<?, ?it/s]

Compiling FSM index for all state transitions:  13%|█▎        | 5/38 [00:00<00:02, 16.14it/s]

Compiling FSM index for all state transitions:  34%|███▍      | 13/38 [00:00<00:00, 26.44it/s]

Compiling FSM index for all state transitions:  55%|█████▌    | 21/38 [00:00<00:00, 26.95it/s]

Compiling FSM index for all state transitions:  79%|███████▉  | 30/38 [00:01<00:00, 33.29it/s]

Compiling FSM index for all state transitions: 100%|██████████| 38/38 [00:01<00:00, 28.07it/s]


[2025-01-02 17:28:39 TP0] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 30, cache hit rate: 37.61%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-02 17:28:39] INFO:     127.0.0.1:37328 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression (use default "outlines" backend)

In [8]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=128,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(response.choices[0].message.content)

[2025-01-02 17:28:39 TP0] Prefill batch. #new-seq: 1, #new-token: 12, #cached-token: 30, cache hit rate: 42.75%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-02 17:28:40] INFO:     127.0.0.1:37328 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF (use "xgrammar" backend)

In [9]:
# terminate the existing server(that's using default outlines backend) for this demo
terminate_process(server_process)

# start new server with xgrammar backend
server_process = execute_shell_command(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0 --grammar-backend xgrammar"
)
wait_for_server("http://localhost:30000")

# EBNF example
ebnf_grammar = r"""
        root ::= "Hello" | "Hi" | "Hey"
        """
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful EBNF test bot."},
        {"role": "user", "content": "Say a greeting."},
    ],
    temperature=0,
    max_tokens=32,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(response.choices[0].message.content)

[2025-01-02 17:28:49] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, return_token_ids=False, host='0.0.0.0', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, random_seed=621760966, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_

[2025-01-02 17:29:03 TP0] Init torch distributed begin.


[2025-01-02 17:29:03 TP0] Load weight begin. avail mem=78.81 GB


[2025-01-02 17:29:04 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-01-02 17:29:14] INFO:     Started server process [1674757]
[2025-01-02 17:29:14] INFO:     Waiting for application startup.
[2025-01-02 17:29:14] INFO:     Application startup complete.
[2025-01-02 17:29:14] INFO:     Uvicorn running on http://0.0.0.0:30000 (Press CTRL+C to quit)


[2025-01-02 17:29:14] INFO:     127.0.0.1:56826 - "GET /v1/models HTTP/1.1" 200 OK


[2025-01-02 17:29:15] INFO:     127.0.0.1:56836 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-01-02 17:29:15 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-02 17:29:16] INFO:     127.0.0.1:56848 - "POST /generate HTTP/1.1" 200 OK
[2025-01-02 17:29:16] The server is fired up and ready to roll!


[2025-01-02 17:29:19 TP0] Prefill batch. #new-seq: 1, #new-token: 48, #cached-token: 1, cache hit rate: 1.79%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-02 17:29:19] INFO:     127.0.0.1:59300 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [10]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-01-02 17:29:19] INFO:     127.0.0.1:59312 - "POST /v1/files HTTP/1.1" 200 OK
[2025-01-02 17:29:19] INFO:     127.0.0.1:59312 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-01-02 17:29:19 TP0] Prefill batch. #new-seq: 2, #new-token: 30, #cached-token: 50, cache hit rate: 37.50%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [11]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-01-02 17:29:20 TP0] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, gen throughput (token/s): 8.35, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2025-01-02 17:29:22] INFO:     127.0.0.1:59312 - "GET /v1/batches/batch_b80fb8e6-8126-44fc-97ca-bea1bb39be3d HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-01-02 17:29:22] INFO:     127.0.0.1:59312 - "GET /v1/files/backend_result_file-8f7fba50-023e-4e75-9f28-9563248983ee/content HTTP/1.1" 200 OK


[2025-01-02 17:29:23] INFO:     127.0.0.1:59312 - "DELETE /v1/files/backend_result_file-8f7fba50-023e-4e75-9f28-9563248983ee HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [12]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(100):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-01-02 17:29:23] INFO:     127.0.0.1:59318 - "POST /v1/files HTTP/1.1" 200 OK
[2025-01-02 17:29:23] INFO:     127.0.0.1:59318 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-01-02 17:29:23 TP0] Prefill batch. #new-seq: 12, #new-token: 360, #cached-token: 300, cache hit rate: 44.10%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-02 17:29:23 TP0] Prefill batch. #new-seq: 88, #new-token: 2640, #cached-token: 2200, cache hit rate: 45.26%, token usage: 0.00, #running-req: 12, #queue-req: 0


[2025-01-02 17:29:23 TP0] Decode batch. #running-req: 100, #token: 5125, token usage: 0.01, gen throughput (token/s): 669.39, #queue-req: 0


[2025-01-02 17:29:23 TP0] Decode batch. #running-req: 100, #token: 9125, token usage: 0.02, gen throughput (token/s): 11906.21, #queue-req: 0


[2025-01-02 17:29:24 TP0] Decode batch. #running-req: 100, #token: 13125, token usage: 0.03, gen throughput (token/s): 11620.72, #queue-req: 0


...

[2025-01-02 17:29:33] INFO:     127.0.0.1:60008 - "GET /v1/batches/batch_28507d67-3fcd-42dd-92fb-7070932a30c1 HTTP/1.1" 200 OK


[2025-01-02 17:29:36] INFO:     127.0.0.1:60008 - "GET /v1/batches/batch_28507d67-3fcd-42dd-92fb-7070932a30c1 HTTP/1.1" 200 OK


[2025-01-02 17:29:39] INFO:     127.0.0.1:60008 - "GET /v1/batches/batch_28507d67-3fcd-42dd-92fb-7070932a30c1 HTTP/1.1" 200 OK


[2025-01-02 17:29:42] INFO:     127.0.0.1:60008 - "GET /v1/batches/batch_28507d67-3fcd-42dd-92fb-7070932a30c1 HTTP/1.1" 200 OK


[2025-01-02 17:29:45] INFO:     127.0.0.1:60008 - "GET /v1/batches/batch_28507d67-3fcd-42dd-92fb-7070932a30c1 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [13]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(500):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-01-02 17:29:48] INFO:     127.0.0.1:47560 - "POST /v1/files HTTP/1.1" 200 OK
[2025-01-02 17:29:48] INFO:     127.0.0.1:47560 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-01-02 17:29:48 TP0] Prefill batch. #new-seq: 12, #new-token: 12, #cached-token: 648, cache hit rate: 50.81%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-02 17:29:48 TP0] Prefill batch. #new-seq: 350, #new-token: 7948, #cached-token: 11302, cache hit rate: 56.76%, token usage: 0.01, #running-req: 12, #queue-req: 0
[2025-01-02 17:29:48 TP0] Prefill batch. #new-seq: 138, #new-token: 4140, #cached-token: 3450, cache hit rate: 54.17%, token usage: 0.02, #running-req: 362, #queue-req: 0


[2025-01-02 17:29:49 TP0] Decode batch. #running-req: 500, #token: 35525, token usage: 0.08, gen throughput (token/s): 931.52, #queue-req: 0


[2025-01-02 17:29:50 TP0] Decode batch. #running-req: 500, #token: 55525, token usage: 0.12, gen throughput (token/s): 26296.73, #queue-req: 0


[2025-01-02 17:29:50 TP0] Decode batch. #running-req: 500, #token: 75525, token usage: 0.17, gen throughput (token/s): 24791.59, #queue-req: 0


...

[2025-01-02 17:29:56 TP0] Decode batch. #running-req: 500, #token: 195525, token usage: 0.44, gen throughput (token/s): 19062.31, #queue-req: 0


[2025-01-02 17:29:57 TP0] Decode batch. #running-req: 500, #token: 215525, token usage: 0.48, gen throughput (token/s): 18319.31, #queue-req: 0


[2025-01-02 17:29:58] INFO:     127.0.0.1:55372 - "POST /v1/batches/batch_c1cbdec9-9fa1-4a2c-9421-23fdf8909a87/cancel HTTP/1.1" 200 OK


[2025-01-02 17:30:01] INFO:     127.0.0.1:55372 - "GET /v1/batches/batch_c1cbdec9-9fa1-4a2c-9421-23fdf8909a87 HTTP/1.1" 200 OK


[2025-01-02 17:30:01] INFO:     127.0.0.1:55372 - "DELETE /v1/files/backend_input_file-46eb243c-d4aa-4b2a-be7d-7291261eb3be HTTP/1.1" 200 OK


In [14]:
terminate_process(server_process)